In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
from peft import PeftModelForSeq2SeqLM, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


peft_model_id = "lukaemon/flan-t5-xxl-financial-phrasebank-lora"

peft_config = PeftConfig.from_pretrained(peft_model_id)

peft_config.base_model_name_or_path


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


'google/flan-t5-xxl'

In [2]:
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

model = AutoModelForSeq2SeqLM.from_pretrained(peft_config.base_model_name_or_path, device_map='auto', load_in_8bit=True)

model = PeftModelForSeq2SeqLM.from_pretrained(model, peft_model_id, device_map={"": 0})

model.print_trainable_parameters()
model.hf_device_map

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

trainable params: 18874368 || all params: 11154206720 || trainable%: 0.16921300163961817


{'': 0}

In [3]:
with torch.cuda.amp.autocast():
    input_text = "In January-September 2009 , the Group 's net interest income increased to EUR 112.4 mn from EUR 74.3 mn in January-September 2008 ."
    inputs = tokenizer(input_text, return_tensors="pt")

    outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=10)

    print("input sentence: ", input_text)
print("output prediction: ", tokenizer.decode(outputs[0], skip_special_tokens=True))


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1370: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
2023-02-21 07:53:53.185681: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-21 07:53:53.185718: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shar

input sentence:  In January-September 2009 , the Group 's net interest income increased to EUR 112.4 mn from EUR 74.3 mn in January-September 2008 .
output prediction:  positive
